# Imports

In [1]:
import numpy as np
import pandas as pd

import plotly.express as px

# Load in Data

In [2]:
POSITION = "qb"

In [3]:
pickle_to_load = f"../../FantasyData/data-frames/df_{POSITION}_college_2000_2023.pkl"

In [4]:
df_pos = pd.read_pickle(pickle_to_load)

### For saving Data Later

In [56]:
save_path = f"../../interactive-2.0/{POSITION.upper()}/draft-capital/"

# Clean Up Data

In [6]:
df_clean = df_pos.copy().dropna(subset=["draft_ovr"])

# Get Draft Capital Success Rate By Round

Grab the best (lower tier is better) tier a player has been in, for each round of the draft:

In [7]:
tier_counts = df_clean.groupby(['draft_round', 'player_name'])['tier'].min().reset_index()

Then group this data frame by the draft round to get the total number of players in each round

In [8]:
total_players_per_round = tier_counts \
    .groupby('draft_round')['player_name'] \
    .count() \
    .reset_index(name='total_players')

Grab the number of each players in each tier, by their draft round:

In [9]:
players_in_tier = tier_counts.groupby(['draft_round', 'tier']).size().reset_index(name='players_in_tier')

Merge the dataframes together:

In [10]:
merged_data = pd.merge(players_in_tier, total_players_per_round, on='draft_round')
merged_data['percentage'] = (merged_data['players_in_tier'] / merged_data['total_players']) * 100

Tier percentagers by draft round:

In [11]:
merged_data.head(5)

,draft_round,tier,players_in_tier,total_players,percentage
0,1.0,1,35,63,55.555556
1,1.0,2,12,63,19.047619
2,1.0,3,12,63,19.047619
3,1.0,4,3,63,4.761905
4,1.0,5,1,63,1.587302


# Analysis

Tier threshold below. Set to 4 for WRs, 2 for QBs.

In [29]:
THRESHOLD = 2

In [30]:
df_success = merged_data.query(f"tier <= {THRESHOLD}").groupby("draft_round").apply(lambda x: x)

/tmp/ipykernel_15172/3481710369.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_success = merged_data.query(f"tier <= {THRESHOLD}").groupby("draft_round").apply(lambda x: x)


In [31]:
df_success.drop(columns="draft_round", inplace=True)

In [32]:
df_success.reset_index(inplace=True)

In [33]:
df_success.rename(columns={"draft_round": "Draft Round", 
                           "percentage": "Success Rate", "players_in_tier": "Sample Size of Tier",
                           "total_players": "All Players in this Round",
                           "tier": "Best Season Tier"
                          }, inplace=True)

In [34]:
df_success.drop(columns="level_1", inplace=True)

In [35]:
df_success.head()

,Draft Round,Best Season Tier,Sample Size of Tier,All Players in this Round,Success Rate
0,1.0,1,35,63,55.555556
1,1.0,2,12,63,19.047619
2,2.0,1,6,18,33.333333
3,2.0,2,4,18,22.222222
4,3.0,1,3,20,15.000000


### Bust DF

In [36]:
df_bust = merged_data.query(f"tier > {THRESHOLD}").groupby("draft_round").agg({"percentage": "sum"})

In [37]:
df_bust.rename(columns={"percentage": "Chance of Bust"}, inplace=True)

In [38]:
df_bust = df_bust.reset_index().rename(columns={'draft_round': 'Draft Round'})

In [39]:
df_bust["Chance of Success"] = 100 - df_bust["Chance of Bust"]

In [40]:
df_bust

,Draft Round,Chance of Bust,Chance of Success
0,1.0,25.396825,74.603175
1,2.0,44.444444,55.555556
2,3.0,60.000000,40.000000
3,4.0,77.272727,22.727273
4,5.0,94.117647,5.882353
5,6.0,77.777778,22.222222
6,7.0,66.666667,33.333333


# Highlighting

In [41]:
# Define a function to highlight rows based on a condition 
def highlight_row(row): 
    # Mapping of pick ranges to background colors
    color_map = {
        1.0: 'yellow',
        2.0: 'lightblue',
        3.0: 'lightgreen',
        4.0: 'lightpink',
        5.0: 'lightgrey',
        6.0: 'lightcoral',
        7.0: 'lightsalmon',
    }
    
    # Get the background color for the current row's pick range
    # Default to white if the pick range is not in the map
    bg_color = color_map.get(row['Draft Round'], 'white')
    
    # Return a list of styles for each column in the row
    return [f'background-color: {bg_color}']*len(row)

In [42]:
df_1 = df_success.style.apply(highlight_row, axis=1)

In [54]:
df_1

,Draft Round,Best Season Tier,Sample Size of Tier,All Players in this Round,Success Rate
0,1.000000,1.000000,35,63,55.555556
1,1.000000,2.000000,12,63,19.047619
2,2.000000,1.000000,6,18,33.333333
3,2.000000,2.000000,4,18,22.222222
4,3.000000,1.000000,3,20,15.000000
5,3.000000,2.000000,5,20,25.000000
6,4.000000,1.000000,3,22,13.636364
7,4.000000,2.000000,2,22,9.090909
8,5.000000,2.000000,1,17,5.882353
9,6.000000,1.000000,3,18,16.666667


In [57]:
with open(f'{save_path}/success-by-round-table.html', 'w') as file:
    file.write(df_1.to_html())

In [45]:
df_2 = df_bust.style.apply(highlight_row, axis=1)

In [58]:
df_2

,Draft Round,Chance of Bust,Chance of Success
0,1.000000,25.396825,74.603175
1,2.000000,44.444444,55.555556
2,3.000000,60.000000,40.000000
3,4.000000,77.272727,22.727273
4,5.000000,94.117647,5.882353
5,6.000000,77.777778,22.222222
6,7.000000,66.666667,33.333333


In [59]:
with open(f'{save_path}/busts-by-round-table.html', 'w') as file:
    file.write(df_2.to_html())

# Plotly Bar Charts

In [69]:
df_success["Best Season Tier"] = df_success["Best Season Tier"].astype(str)

In [71]:
fig = px.bar(df_success, x="Draft Round", y="Success Rate",
             color='Best Season Tier', barmode='group',
             hover_data=['Sample Size of Tier', 'All Players in this Round'],
             )

fig.write_html(f"{save_path}/hit-rate-v-round.html")
fig.show()

In [73]:
df_success["Draft Round"] = df_success["Draft Round"].astype(str)

In [74]:
fig = px.bar(df_success, x="Best Season Tier", y="Success Rate",
             color='Draft Round', barmode='group',
             hover_data=['Sample Size of Tier', 'All Players in this Round'],
             )

fig.write_html(f"{save_path}/hit-rate-v-tier-by-round.html")
fig.show()

In [62]:
fig = px.bar(df_bust, x="Draft Round", y="Chance of Bust")

fig.write_html(f"{save_path}/bust-rate-v-round.html")
fig.show()